In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import zipfile

# Specify the path to the zip archive
zip_path = '/content/drive/MyDrive/3dGrazNormalized.zip'

# Specify the directory where you want to extract the files
extract_dir = '/content/'

# Open the zip archive
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # Extract all the contents of the zip archive to the specified directory
    zip_ref.extractall(extract_dir)

print("Folder decompressed successfully.")

Folder decompressed successfully.


In [3]:
!pip install braindecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.4/184.4 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 84.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 19.1 MB/s eta 0:00:00


In [8]:
import torch
import torch.nn as nn

# Create the Conv3d layer
conv = nn.Conv3d(1, 40, (1, 1, 25), stride=(1, 1, 1), padding=(0, 0, 0))

# Create a sample input tensor
input_tensor = torch.randn(40, 1, 6, 7, 1000)  # Adjust the input shape as needed

# Pass the input tensor through the Conv3d layer
output_tensor = conv(input_tensor)
print("Output shape after Conv3d:", output_tensor.shape)


Output shape after Conv3d: torch.Size([40, 40, 6, 7, 976])


In [9]:
conv = nn.Conv3d(40, 40, (6, 7, 1), stride=(1,1,1), padding=(0,0,0))

# Create a sample input tensor
input_tensor = torch.randn(40, 40, 6, 7, 976)  # Adjust the input shape as needed

# Pass the input tensor through the Conv3d layer
output_tensor = conv(input_tensor)
print("Output shape after Conv3d:", output_tensor.shape)

Output shape after Conv3d: torch.Size([40, 40, 1, 1, 976])


In [ ]:
import scipy.io
import numpy as np
from skorch.helper import predefined_split
from sklearn.model_selection import train_test_split
from braindecode.models import Deep4Net
from braindecode import EEGClassifier
import torch
import os
import scipy.io
from skorch.callbacks import LRScheduler
from braindecode import EEGClassifier
import torch.nn as nn
import torch.optim as optim

data_dir = '/content/3dGrazNormalized/'


class ModifiedDeep4Net(nn.Module):
    def __init__(self):
        super(ModifiedDeep4Net, self).__init__()
        self.shallownet1 = nn.Sequential(
            nn.Conv3d(1, 40, (1, 1, 50), stride=(1,1,1), padding=(0,0,0)),
            nn.Conv3d(40, 40, (6, 7, 1), stride=(1,1,1), padding=(0,0,0)),
        )
        self.shallownet2 = nn.Sequential(
            nn.BatchNorm2d(40),
        )
        self.deep4net = Deep4Net(
            in_chans=40,  # Number of input channels
            n_classes=4,  # Number of classes
            input_window_samples=951,
            final_conv_length='auto'
        )

    def forward(self, x):
        x = self.shallownet1(x)
        x = torch.squeeze(x, dim=2)
        x = self.shallownet2(x)
        x = x.permute(0, 1, 3, 2)
        x = self.deep4net(x)
        return x

# Create an instance of the modified model
model = ModifiedDeep4Net()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 100
batch_size = 32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


best_valid_accuracy = []
aver_valid_accuracy = []
for i in range(1,10):
    log_write = open("log_subject%d.txt" % i, "w")

    file_name1 = f'3Dnorm_2d_{i:02d}T_P.mat'  # Construct the file name
    file_name2 = f'3Dnorm_2d_{i:02d}E_P.mat'  # Construct the file name


    file_path1 = os.path.join(data_dir, file_name1)  # Construct the file path
    file_path2 = os.path.join(data_dir, file_name2)  # Construct the file path
    training = scipy.io.loadmat(file_path1)
    testing = scipy.io.loadmat(file_path2)

    # Assuming you have your training and testing datasets
    train_X, train_y = training['data'], (np.squeeze(training['label'])).astype(int)-1
    valid_X, valid_y = testing['data'], (np.squeeze(testing['label'])).astype(int)-1

    # Create the dataset objects
    train_dataset = torch.utils.data.TensorDataset(torch.Tensor(train_X), torch.Tensor(train_y))
    valid_dataset = torch.utils.data.TensorDataset(torch.Tensor(valid_X), torch.Tensor(valid_y))

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    best_valid_acc = 0
    aver_valid_acc = 0
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_acc = 0.0

        for inputs, labels in train_loader:
            inputs = torch.unsqueeze(inputs, dim=1).to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.long())
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs,1)
            train_acc += torch.sum(preds == labels.data)

        epoch_loss = train_loss / len(train_dataset)
        epoch_acc = train_acc.double() / len(train_dataset)

        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f"Train Loss: {epoch_loss:.4f} | Train Accuracy: {epoch_acc:.4f}")

        model.eval()
        valid_loss = 0.0
        valid_acc = 0.0

        with torch.no_grad():
            for inputs, labels in valid_dataset:
                inputs = inputs.unsqueeze(0).unsqueeze(0).to(device)
                labels = labels.unsqueeze(0).to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels.long())

                valid_loss += loss.item() * inputs.size(0)
                _,preds = torch.max(outputs, 1)
                valid_acc += torch.sum(preds == labels.data)

        valid_loss = valid_loss / len(valid_dataset)
        valid_acc = valid_acc.double() / len(valid_dataset)

        print(f"Validation Loss: {valid_loss:.4f} | Validation Accuracy: {valid_acc:.4f}")

        if best_valid_acc < valid_acc:
            best_valid_acc = valid_acc
        aver_valid_acc += valid_acc

        log_write.write(str(epoch) + "    " +f"Train Loss: {epoch_loss:.4f} | Train Accuracy: {epoch_acc:.4f}  |  Validation Loss: {valid_loss:.4f} | Validation Accuracy: {valid_acc:.4f}"+ "\n")

    best_valid_accuracy.append(best_valid_acc)
    aver_valid_accuracy.append(aver_valid_acc/num_epochs)

    log_write.write('The average accuracy is: ' + str(aver_valid_acc/num_epochs) + "\n")
    log_write.write('The best accuracy is: ' + str(best_valid_acc) + "\n")

result_write = open("sub_result.txt", "w")
for i in range(len(best_valid_accuracy)):
    result_write.write('Subject ' + str(i + 1) + ' : ' + 'The best accuracy is: ' + str(best_valid_accuracy[i]) + "\n")
    result_write.write('Subject ' + str(i + 1) + ' : ' + 'The average accuracy is: ' + str(aver_valid_accuracy[i]) + "\n")

best = sum(best_valid_accuracy) /9
aver = sum(aver_valid_accuracy) /9
result_write.write('**The average Best accuracy is: ' + str(best) + "\n")
result_write.write('The average Aver accuracy is: ' + str(aver) + "\n")
result_write.close()


Epoch 1/100
Train Loss: 1.6867 | Train Accuracy: 0.3153
Validation Loss: 1.6653 | Validation Accuracy: 0.2768
Epoch 2/100
Train Loss: 1.6376 | Train Accuracy: 0.3477
Validation Loss: 1.6213 | Validation Accuracy: 0.3125
Epoch 3/100
Train Loss: 1.4199 | Train Accuracy: 0.4557
Validation Loss: 1.7100 | Validation Accuracy: 0.3661
Epoch 4/100
Train Loss: 1.2820 | Train Accuracy: 0.4579
Validation Loss: 2.0163 | Validation Accuracy: 0.3304


In [ ]:
import shutil

# Specify the source directory where the text files are located
source_directory = '/content/'

# Specify the destination directory in your Google Drive
destination_directory = '/content/drive/MyDrive/result3dconv-cnngraz'

# Move all text files from the source directory to the destination directory
for file_name in os.listdir(source_directory):
    if file_name.endswith('.txt'):
        file_path = os.path.join(source_directory, file_name)
        shutil.move(file_path, destination_directory)

print("Text files moved to Google Drive successfully.")